In [28]:
# !git clone https://github.com/facebookresearch/dinov2.git

In [37]:
# %cd /home/kareem/Desktop/dinv2_Image_clustering/dinov2

In [38]:
# pip install -r requirements.txt

In [39]:
import torch
import numpy as np 
import matplotlib.pyplot as plt 
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,utils
from torchvision.datasets import ImageFolder
import warnings 
warnings.filterwarnings("ignore")

from PIL import Image
import torchvision.transforms as T
# import hubconf

dino = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
#! Change the path to your query image 
test_img = Image.open('/home/kareem/Desktop/dinv2_Image_clustering/_.jpeg')

image_transforms = T.Compose([
    T.Resize(256, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])



t_img = image_transforms(test_img)
print(t_img.shape)
t_img = t_img.unsqueeze(0)
print(t_img.shape)

Using cache found in /home/kareem/.cache/torch/hub/facebookresearch_dinov2_main


torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])


In [40]:
torch.cuda.is_available()

True

In [41]:
dino = dino.cuda()
t_img = t_img.cuda()
dino_emb = dino(t_img)

In [42]:
dino_emb.size()

torch.Size([1, 768])

# Search 

In [43]:
# !pip install faiss-gpu
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
import faiss

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

query_image_path = "/home/kareem/Desktop/dinv2_Image_clustering/dataset/train/21/image_06783.jpg"
query_image = Image.open(query_image_path)
query_image.show()
query_image = image_transforms(query_image).unsqueeze(0).to(device)
query_image.size()
dino.eval()
with torch.no_grad():
    query_features = dino(query_image).float()
dataset_path = "/home/kareem/Desktop/dinv2_Image_clustering/dataset/test"
dataset_images = []
dataset_features = []
for filename in os.listdir(dataset_path):
    image_path = os.path.join(dataset_path,filename)
    image = Image.open(image_path)
    image = image_transforms(image).unsqueeze(0).to(device)
    dataset_images.append(filename)
    with torch.no_grad():
        features = dino(image).float()
    dataset_features.append(features)

# Convert the feature vectors to a Faiss index
index = faiss.IndexFlatL2(query_features.shape[1])
dataset_features = torch.cat(dataset_features, dim=0).cpu().numpy()
index.add(dataset_features)

# Perform nearest neighbor search on the index
k = 5  # number of neighbors to retrieve
distances, indices = index.search(query_features.cpu().numpy(), k)
query_image
for i in range(k):
    image_path = os.path.join(dataset_path, dataset_images[indices[0][i]])
    image = Image.open(image_path)
    image.show()